In [1]:
import warnings
warnings.filterwarnings('ignore')
import os, sys
from ast import literal_eval as make_tuple
from time import time
from pathlib import Path
from itertools import product
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import spearmanr
import seaborn as sns

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
sys.path.append(r'C:\Users\ftiag\Desktop\Business, trading et investissement\Business\modules')
from research_tools import rank_correl, MultipleTimeSeriesCV
from tensorflow_research_tools import make_model

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


In [3]:
np.random.seed(42)
sns.set_style('whitegrid')
idx = pd.IndexSlice

In [4]:
with pd.HDFStore('data.h5') as hdf:
    data = hdf['most_liquid_pairs_data']

In [5]:
data.index.names = ['symbol', 'date']

In [6]:
X = data.drop(['target_1p'], axis=1)
y = data['target_1p']

In [21]:
train_period_length = 300
test_period_length = 30
n_splits = int(len(data.index.get_level_values('date').unique())/test_period_length-train_period_length/test_period_length-1)
lookahead = 1

cv = MultipleTimeSeriesCV(n_splits=n_splits,
                          train_period_length=train_period_length,
                          test_period_length=test_period_length,
                          lookahead=lookahead)

In [22]:
dense_layer_opts = [(8, 4)]
activation_opts = ['elu']
dropout_opts = [0]
batch_size = [256]
epochs = [3]

In [23]:
param_grid = list(product(dense_layer_opts, activation_opts, dropout_opts, batch_size, epochs))
np.random.shuffle(param_grid)

In [24]:
param_grid

[((8, 4), 'elu', 0, 256, 3)]

In [25]:
ic = []
scaler = StandardScaler()
for i, params in enumerate(param_grid, start=1):
    print(i)
    dense_layers, activation, dropout, batch, epoch = params
    predictions = []
    for fold, (train_idx, test_idx) in enumerate(cv.split(X), start=1):
        X_train, y_train = X.iloc[train_idx, :], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx, :], y.iloc[test_idx]
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)
        model = make_model(dense_layers, activation, dropout, X)
        model.fit(X_train, y_train, batch_size=batch, epochs=epoch)
        preds = model.predict(X_test)
        actual_preds = y_test.to_frame('actuals').assign(predicted=preds)
        predictions.append(actual_preds)
    predictions = pd.concat(predictions, axis=0)
    params_ic = [dense_layers, activation, dropout, batch, epoch, spearmanr(predictions.actuals, predictions.predicted)[0]]
    ic.append(params_ic)

1
Epoch 1/3
9/9 [==============================] - 0s 3ms/step - loss: 1.0594
Epoch 2/3
9/9 [==============================] - 0s 3ms/step - loss: 0.7456
Epoch 3/3
9/9 [==============================] - 0s 3ms/step - loss: 0.5370
Epoch 1/3
9/9 [==============================] - 0s 3ms/step - loss: 2.7010
Epoch 2/3
9/9 [==============================] - 0s 3ms/step - loss: 1.8939
Epoch 3/3
9/9 [==============================] - 0s 3ms/step - loss: 1.3666
Epoch 1/3
9/9 [==============================] - 0s 3ms/step - loss: 1.4904
Epoch 2/3
9/9 [==============================] - 0s 3ms/step - loss: 1.0164
Epoch 3/3
9/9 [==============================] - 0s 3ms/step - loss: 0.7125
Epoch 1/3
9/9 [==============================] - 0s 3ms/step - loss: 1.9891
Epoch 2/3
9/9 [==============================] - 0s 3ms/step - loss: 1.3667
Epoch 3/3
9/9 [==============================] - 0s 3ms/step - loss: 0.9652
Epoch 1/3
9/9 [==============================] - 0s 3ms/step - loss: 0.6167
Epoch 2/3


In [26]:
ic = pd.DataFrame(ic, columns=['dense_layers', 'activation', 'dropout', 'batch', 'epoch', 'IC'])

In [29]:
ic.sort_values(by='IC')

,dense_layers,activation,dropout,batch,epoch,IC
0,"(8, 4)",elu,0,256,3,0.000559


In [31]:
with pd.HDFStore('data.h5') as hdf:
    tt = hdf['first_pred']

In [33]:
tickers = data.index.levels[0]

In [34]:
spearmanr(tt.loc[tickers].predicted, tt.loc[tickers].actuals)

SignificanceResult(statistic=0.015212545403319956, pvalue=9.971444032935918e-37)